# Домашнее задание №3. DSSM модель
### Доработайте DSSM модель, добавив temporal слои, то есть учитывающие последовательности айтемов. (8 баллов)

## Импорты

In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Считаем препроцессные данные из прошлого ноутбучка

In [2]:
interactions_df = pd.read_csv('/kaggle/input/kion-train/interactions_processed_kion.csv')
users_df = pd.read_csv('/kaggle/input/kion-train/users_processed_kion.csv')
items_df = pd.read_csv('/kaggle/input/kion-train/items_processed_kion.csv')

In [3]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [4]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

In [5]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [6]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [7]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [8]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [9]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating', 
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Сделаем матрицу взаимодействий

In [10]:
interactions_df.item_id.value_counts()

10440    202457
15297    193123
9728     132865
13865    122119
4151      91167
          ...  
8076          1
8954          1
15664         1
818           1
10542         1
Name: item_id, Length: 15706, dtype: int64

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

Можете не фильтровать такие события, тогда у вас будет больше негативных примеров.

In [11]:
interactions_df.user_id.value_counts()

416206     1341
1010539     764
555233      685
11526       676
409259      625
           ... 
45493         1
615194        1
96848         1
425823        1
697262        1
Name: user_id, Length: 962179, dtype: int64

In [12]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [13]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [14]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


Отнормируем матрицу взаимодействий

In [15]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), 
                             interactions_df.iid.nunique())) 

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [16]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6901
65974
65974
{11805, 9788, 11501, 1734}


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [17]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [18]:
unique_items = interactions_df.item_id.unique()

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(unique_items)]

print(items_ohe_df.item_id.nunique())
print(interactions_df.item_id.nunique())

6897
6897


И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [19]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [20]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

Попробуйте другие лоссы, например, BPR Triplet loss

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [21]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        
        for _ in range(batch_size):
            # выбираем рандомного uid
            uid_i = np.random.randint(0, interactions.shape[0])
            # распределение просмотров айтемов для данного юзера
            interaction_probs = interactions[uid_i]
            # выбираем "хороший" айтем на основе распределения просмотров
            pos_i = np.random.choice(range(interactions.shape[1]), p=interaction_probs)
            # выбираем "плохой" айтем на основе взвешенного среднего распределения
            neg_probs = (1 - interaction_probs) / (interactions.shape[1] - 1)
            neg_i = np.random.choice(range(interactions.shape[1]), p=neg_probs)
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи "хорошего" айтема
            pos.append(items.iloc[pos_i])
            # фичи "плохого" айтема
            neg.append(items.iloc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [22]:
# инициализируем генератор
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                users=users_ohe_df.drop(["user_id"], axis=1), 
                interactions=interactions_vec)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 8588)
вектор 'плохого' айтема: (1024, 8588)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)


### Сделайте генаратор, который будет использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования

In [23]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8588,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [34]:
def item_model(n_factors=N_FACTORS):
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                      kernel_regularizer=keras.regularizers.l2(1e-6),
                                      activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                             kernel_regularizer=keras.regularizers.l2(1e-6),
                                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                      kernel_regularizer=keras.regularizers.l2(1e-6),
                                      activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    
    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    concat_meta_interaction = keras.layers.Reshape((-1, N_FACTORS * 2))(concat_meta_interaction)
    
    lstm_1 = keras.layers.LSTM(N_FACTORS, return_sequences=True)(concat_meta_interaction)
    lstm_2 = keras.layers.LSTM(N_FACTORS)(lstm_1)
    
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(lstm_2)
    
    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [35]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [36]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [37]:
# модель айтема
item_model().summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 8588)]       0           []                               
                                                                                                  
 dense_25 (Dense)               (None, 128)          1099264     ['input_17[0][0]']               
                                                                                                  
 dense_26 (Dense)               (None, 128)          16384       ['dense_25[0][0]']               
                                                                                                  
 add_8 (Add)                    (None, 128)          0           ['dense_25[0][0]',               
                                                                  'dense_26[0][0]']         

In [38]:
# модель юзера
user_model().summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 19)]         0           []                               
                                                                                                  
 dense_28 (Dense)               (None, 128)          2432        ['input_18[0][0]']               
                                                                                                  
 dense_30 (Dense)               (None, 128)          16384       ['dense_28[0][0]']               
                                                                                                  
 input_19 (InputLayer)          [(None, 6897)]       0           []                               
                                                                                            

In [39]:
# общая модель
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 19)]         0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 6897)]       0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 8588)]       0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 8588)]       0           []                               
                                                                                            

In [40]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                    users=users_ohe_df.drop(["user_id"], axis=1), 
                    interactions=interactions_vec,
                    batch_size=64), 
          steps_per_epoch=100, 
          epochs=30, 
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 33s 194ms/step - loss: 0.2726 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 19s 196ms/step - loss: 0.2067 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 19s 196ms/step - loss: 0.1885 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 19s 196ms/step - loss: 0.1875 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 19s 191ms/step - loss: 0.1803 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 19s 196ms/step - loss: 0.1674 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 19s 193ms/step - loss: 0.1729 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 19s 190ms/step - loss: 0.1635 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 20s 200ms/step - loss: 0.1568 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 19s 190ms/step - loss: 0.1598 - lr: 0.0010
Epoch 11/30
100/100 [========

In [41]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), 
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 71ms/step


array([[1.168908]], dtype=float32)

In [42]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

216/216 [==============================] - 0s 2ms/step


In [43]:
items_vecs.shape

(6897, 128)

In [44]:
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)
users_interaction_vec = interactions_vec

In [45]:
users_meta_feats.shape

(65974, 19)

In [46]:
users_interaction_vec.shape

(65974, 6897)

In [47]:
np.array(users_meta_feats).shape

(65974, 19)

In [ ]:
users_vec = u2v.predict([np.array(users_meta_feats), 
                        np.array(users_interaction_vec)])

In [53]:
users_vec.shape

(40329, 128)

In [54]:
items_vecs.shape

(5001, 128)

In [55]:
dists = ED(users_vec, items_vecs)

In [56]:
dists.shape

(40329, 5001)

In [57]:
top10_iids = np.argsort(dists, axis=1)[:,:10]

In [58]:
top10_iids.reshape(dists.shape[0], 10).shape

(40329, 10)

In [59]:
top10_iids

array([[1419, 3108, 2667, ...,  194, 2309, 3527],
       [2941, 4448, 1326, ..., 2137, 1154, 4380],
       [4641,   32, 3144, ..., 1099, 2245, 4434],
       ...,
       [ 194, 1891, 2034, ..., 3226, 4096, 4171],
       [2941, 4448, 1099, ..., 2137, 3686, 4066],
       [4529, 1062, 1108, ..., 1914, 3178,  753]])

In [60]:
top10_iids.shape

(40329, 10)

In [61]:
top10_iids_item = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]

In [62]:
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)

In [63]:
top10_iids_item.shape

(40329, 10)

In [64]:
df_dssm = pd.DataFrame(columns = ['user_id', 'item_id'])

In [65]:
df_dssm.head()

,user_id,item_id


In [66]:
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in np.arange(top10_iids_item.shape[0])]})

In [67]:
df_dssm['item_id'] = list(top10_iids_item)

In [68]:
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

In [69]:
df_dssm.head()

,user_id,item_id
0,2,"[4718, 10323, 8821, 1267, 2301, 16166, 16291, ..."
1,21,"[9728, 14703, 4457, 3734, 5287, 10440, 14741, ..."
2,53,"[15297, 142, 10440, 12192, 8636, 9728, 14703, ..."
3,60,"[10440, 3734, 4151, 14856, 7829, 7417, 3784, 3..."
4,195,"[10440, 3734, 13668, 657, 14431, 12356, 9728, ..."


In [72]:
submission = pd.read_csv('/content/sample_submission.csv')

In [73]:
df_dssm_subm = df_dssm[df_dssm['user_id'].isin(submission['user_id'])]

In [77]:
df_dssm_subm.head()

,user_id,item_id
6,241,"[741, 3808, 15266, 6774, 4718, 10323, 7310, 10..."
7,243,"[9728, 2301, 14317, 8821, 14488, 3734, 10942, ..."
9,322,"[3734, 9728, 10440, 11237, 3784, 4495, 14675, ..."
10,459,"[10440, 3734, 9728, 15297, 142, 14703, 7417, 8..."
11,513,"[7382, 3587, 8821, 5754, 3095, 7210, 13713, 83..."


In [78]:
df_dssm_subm = df_dssm_subm.explode('item_id')
df_dssm_subm['rank'] = df_dssm_subm.groupby('user_id').cumcount() + 1

In [79]:
df_dssm_subm.head()

,user_id,item_id,rank
6,241,741,1
6,241,3808,2
6,241,15266,3
6,241,6774,4
6,241,4718,5


In [80]:
class PopularRecommender():
    def __init__(self, max_K=10, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [83]:
interactions_df = pd.read_csv('/content/interactions_processed_kion.csv', parse_dates=['last_watch_dt'])

In [85]:
from itertools import islice, cycle

In [86]:
train = interactions_df
test = submission

pop_model = PopularRecommender(days=10, dt_column='last_watch_dt')
pop_model.fit(train)

recs = pd.DataFrame({'user_id': test['user_id'].unique()})
recs['item_id'] = pop_model.recommend(recs['user_id'], N=10)
recs = recs.explode('item_id')
recs['rank'] = recs.groupby('user_id').cumcount() + 1
#recs = recs.groupby('user_id').agg({'item_id': list}).reset_index()

In [87]:
recs.head()

,user_id,item_id,rank
0,3,9728,1
0,3,10440,2
0,3,15297,3
0,3,13865,4
0,3,12360,5


In [91]:
recs_pop = recs[~recs['user_id'].isin(df_dssm_subm['user_id'].tolist())]

In [107]:
recs_all = pd.concat([recs_pop, df_dssm_subm], axis = 0)

In [108]:
recs_all.shape

(1931130, 3)

In [112]:
recs_all.head()

,user_id,item_id,rank
0,3,9728,1
0,3,10440,2
0,3,15297,3
0,3,13865,4
0,3,12360,5
